In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=1234")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    conn.close()
    cur.execute("DROP DATABASE wealth_acc")
    cur.execute("CREATE DATABASE wealth_acc")
    conn.close()
    conn = psycopg2.connect("host=localhost dbname=wealth_acc user=postgres password=1234")
    cur = conn.cursor()
    return conn,cur

def db_connect():
    conn = psycopg2.connect("host=localhost dbname=wealth_acc user=postgres password=1234")
    cur = conn.cursor()
    return conn,cur

In [3]:
try :
    acc_country = pd.read_csv("E:\datasets\Wealth_Acc\Wealth-AccountsCountry.csv")
    acc_data = pd.read_csv("E:\datasets\Wealth_Acc\Wealth-AccountData.csv")
    acc_series = pd.read_csv("E:\datasets\Wealth_Acc\Wealth-AccountSeries.csv")
except pd.Error as e:
    print(e)
 
acc_country = acc_country[['Code','Short Name','Long Name','Currency Unit','Table Name']]
acc_country.head()


In [4]:
# acc_data.columns
acc_data = acc_data[['Country Code','Country Name','Series Code','Series Name','1995 [YR1995]','2000 [YR2000]','2005 [YR2005]','2010 [YR2010]','2015 [YR2015]']]
acc_data.head()

In [5]:
acc_series = acc_series[['Code','Topic','Indicator Name', 'Long definition']]
acc_series.head()

In [7]:
conn, cur = create_database()
#conn, cur = db_connect()

In [8]:
cur.execute("DROP TABLE IF EXISTS accountscountry")
cur.execute("DROP TABLE IF EXISTS accountsdata")
cur.execute("DROP TABLE IF EXISTS accountsseries")


In [9]:
acc_country_tbl_creation = ('''CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
long_name VARCHAR,
table_name VARCHAR,
currency_unit VARCHAR)''')

acc_data_tbl_creation = ('''CREATE TABLE IF NOT EXISTS accountsdata(
Country_Code VARCHAR,
Country_Name VARCHAR,
Series_Code VARCHAR,
Series_Name VARCHAR,
YR1995 NUMERIC,
YR2000 NUMERIC,
YR2005 NUMERIC,
YR2010 NUMERIC,
YR2015 NUMERIC)''')

acc_series_tbl_creation = ('''CREATE TABLE IF NOT EXISTS accountsseries(
series_Code VARCHAR,
Topic VARCHAR,
Indicator_Name VARCHAR,
Long_definition VARCHAR
)''')

In [10]:
cur.execute(acc_country_tbl_creation)
cur.execute(acc_data_tbl_creation)
cur.execute(acc_series_tbl_creation)
conn.commit()


In [11]:
acc_country_insert_script = ('''INSERT INTO accountscountry(
country_code,
short_name,
long_name,
table_name,
currency_unit)
VALUES (%s,%s,%s,%s,%s)
''')

acc_data_insert_script = ('''INSERT INTO accountsdata(
Country_Code,
Country_Name,
Series_Code,
Series_Name,
YR1995,
YR2000,
YR2005,
YR2010,
YR2015)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
''')

acc_series_insert_script = ('''INSERT INTO accountsseries(
series_Code,
Topic,
Indicator_Name,
Long_definition)
VALUES (%s,%s,%s,%s)''')

In [12]:
count =0
for i,row in acc_country.iterrows():
    #print(list(row))
    #count += 1
    #print(count)
    cur.execute(acc_country_insert_script,list(row))

for i,row in acc_data.iterrows():
    cur.execute(acc_data_insert_script,list(row))
    
for i,row in acc_series.iterrows():
    cur.execute(acc_series_insert_script,list(row))

conn.commit()

In [23]:
cur.execute("SELECT * FROM accountscountry;")
rows = cur.fetchall()

for row in rows:
    print(row)
 
conn.commit()

In [21]:
cur.close()
conn.close()